In [49]:
# Import required libraries
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
import itertools
import wandb
import argparse
import random

# QOOP-specific imports
from qoop.evolution import normalizer
from qoop.evolution.environment_synthesis import MetadataSynthesis
from qoop.evolution.generator import by_num_rotations_and_cnot
from qoop.evolution.environment import EEnvironment
from qoop.evolution.crossover import onepoint
from qoop.evolution.mutate import bitflip_mutate_with_normalizer
from qoop.evolution.threshold import synthesis_threshold
from qoop.backend.constant import operations_with_rotations
from qoop.evolution import divider


from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [72]:
from qiskit.qpy import dump, load

with open('/home/qsvm/GA-QSVM/7qubits_train_qsvm_2025-06-16/circuit_1_2.qpy', 'rb') as f:
    loaded_circuits = load(f) # qpy.load returns a list of circuits

qc_loaded = loaded_circuits[0]
qc_loaded.draw()

┌──────────────┐┌──────────────┐┌──────────────┐
q_0: ┤ Ry(theta[1]) ├┤ Ry(theta[3]) ├┤ Ry(theta[5]) ├
     └────┬───┬─────┘└──────────────┘└──────────────┘
q_1: ─────┤ H ├──────────────────────────────────────
          └───┘                                      
q_2: ────────────────────────────────────────────────
          ┌───┐      ┌──────────────┐                
q_3: ─────┤ H ├──────┤ Ry(theta[2]) ├────────────────
     ┌────┴───┴─────┐├──────────────┤┌──────────────┐
q_4: ┤ Ry(theta[0]) ├┤ Ry(theta[4]) ├┤ Ry(theta[6]) ├
     └────┬───┬─────┘└──────────────┘└──────────────┘
q_5: ─────┤ H ├──────────────────────────────────────
          └───┘                                      
q_6: ────────────────────────────────────────────────

In [51]:
print(f"Number of qubits in loaded circuit: {qc_loaded.num_qubits}")
print(f"Number of parameters in loaded circuit: {qc_loaded.num_parameters}")
print("Parameters:", qc_loaded.parameters) # Shows the Parameter objects

Number of qubits in loaded circuit: 7
Number of parameters in loaded circuit: 8
Parameters: ParameterView([Parameter(0), Parameter(5), Parameter(theta(0)), Parameter(theta(1)), Parameter(theta(2)), Parameter(theta(3)), Parameter(theta(4)), Parameter(theta(5))])


In [52]:
from sklearn.datasets import load_wine, load_digits, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import numpy as np
# Load cancer data using the function from data/split.py
# Number of features should match the number of qubits in the quantum circuit
n_features = 7  # Based on the 7-qubit circuit loaded earlier
training_size = 100
test_size = 100


def prepare_cancer_data_split(training_size, test_size, n_features, random_state):
    digits = load_breast_cancer()
    X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=100,train_size=100, random_state=random_state, stratify=digits.target)
    
    # Scale the features 
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Reduce dimensionality using PCA
    pca = PCA(n_components=n_features)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    print(f"Training size: {len(X_train)}, Test size: {len(X_test)}")

    return X_train, X_test, y_train, y_test 

In [53]:
for i in range(100, 110):

    # Load and prepare cancer dataset
    X_train, X_test, y_train, y_test = prepare_cancer_data_split(
        training_size=training_size, 
        test_size=test_size, 
        n_features=7, 
        random_state=i
    )

    # Training function adapted from main.py to work with our loaded data
    def train_qsvm(quantum_circuit):
        """
        Train Quantum SVM using the cancer dataset
        
        Args:
            quantum_circuit: Quantum circuit to use as feature map
        
        Returns:
            Classification accuracy
        """
        quantum_kernel = FidelityQuantumKernel(feature_map=quantum_circuit)
        qsvc = QSVC(quantum_kernel=quantum_kernel)
        qsvc.fit(X_train, y_train)
        y_pred = qsvc.predict(X_test)
        return accuracy_score(y_test, y_pred)

    # Train the QSVM with the loaded quantum circuit
    print("Training Quantum SVM with the loaded 7-qubit circuit...")
    accuracy = train_qsvm(qc_loaded)
    print(f"Classification accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")


Training size: 100, Test size: 100
Training Quantum SVM with the loaded 7-qubit circuit...


ValueError: x_vec and class feature map have incompatible dimensions.
x_vec has 7 dimensions, but feature map has 8.